<div style="width: 100%; clear: both;">
    <div style="float: left; width: 50%;">
        <img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg" align="left">
    </div>
</div>
<div style="float: right; width: 50%;">
    <p style="margin: 0; padding-top: 22px; text-align:right;">M2.876 · Análisis de grafos y redes sociales</p>
    <p style="margin: 0; text-align:right;">Máster universitario en Ciencias de Datos (<i>Data Science</i>)</p>
    <p style="margin: 0; text-align:right; padding-button: 100px;">Estudios de Informática, Multimedia y Telecomunicaciones</p>
</div>
<div style="width: 100%; clear: both;"></div>
<div style="width:100%;">&nbsp;</div>

# Hipergrafos y complejos simpliciales

En este _notebook_ revisaremos y aplicaremos los conocimientos relacionados con los hipergrafos y los complejos simpliciales.

## Carga de librerías

En la siguiente celda se deben cargar todas las librerías necesarias para ejecutar este ejemplo.

In [ ]:
# Librerías básicas
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import random

%matplotlib inline

Las versiones de librerías que se deben emplear en esta actividad son las siguientes:
- NetworkX ver. 2.8.6
- Numpy ver. 1.23.3

In [ ]:
# Comprobar las versiones
print("NetworkX ver. {}".format(nx.__version__))
print("Numpy    ver. {}".format(np.__version__))

NetworkX ver. 2.8.4
Numpy    ver. 1.21.5


## 1. Carga del conjunto de datos

En esta actividad vamos a trabajar con un conjunto de datos que proviene de dos redes diferentes. Este conjunto de datos, que se encuentra disponible en la carpeta `data` adjunta al enunciado, está formado por los ficheros:

- `layer1.edgelist`: contiene todas las aristas y sus respectivos pesos de un layer de nuestra red. 
- `layer2.edgelist`: contiene todas las aristas y sus respectivos pesos de un layer de nuestra red.

### 1.1. Importación de los datos y creación del grafo

Usando los dos layers proporcionados, correspondientes a una red multiplex, construiremos un grafo **simétrico** (no dirigido) y **ponderado**. 

En caso DE que una arista se encuentre en ambos layers, su peso asociado será la suma de los pesos de cada layer. 

Empleamos la función `read_edgelist` de la librería `NetworkX` para implementar la lectura y carga de las capas (_layers_) y generación de la red.

El formato de los ficheros de entrada es el siguiente:
- 0 1 {'weight': 0.3}
- 0 2 {'weight': 0.4}
- 0 3 {'weight': 0.2}

Es decir, línea del fichero de entrada representa una arista en el siguiente formato: 
- `nodo origen` `nodo destino` {'weight': `peso arista`}

In [ ]:
file_layer1 = "data/layer1.edgelist"
file_layer2 = "data/layer2.edgelist"

F1 = nx.read_edgelist(file_layer1)
F2 = nx.read_edgelist(file_layer2)

print('layer1: num nodes {}, num edges {}'.format(nx.number_of_nodes(F1),nx.number_of_edges(F1)))
print('layer2: num nodes {}, num edges {}'.format(nx.number_of_nodes(F2),nx.number_of_edges(F2)))

layer1: num nodes 100, num edges 930
layer2: num nodes 98, num edges 260


In [ ]:
print('nodes intersection: {}'.format(len(set(F1.nodes()).intersection(set(F2.nodes())))))
print('edges intersection: {}'.format(len(set(F1.edges()).intersection(set(F2.edges())))))

nodes intersection: 78
edges intersection: 0


Podemos observar que el número de nodos no es el mismo en ambas redes, y que el número de aristas difiere sensiblemente.

A partir de los datos de ambas capas, crearemos la red multiplex en el siguiente código:

In [ ]:
# join network
edg1 = [(a,b,c['weight']) for a,b,c in F1.edges(data=True)]
edg2 = [(a,b,c['weight']) for a,b,c in F2.edges(data=True)]

G = nx.Graph()
G.add_weighted_edges_from(edg1)
G.add_weighted_edges_from(edg2)

print('G: num nodes {}, num edges {}'.format(nx.number_of_nodes(G),nx.number_of_edges(G)))

G: num nodes 120, num edges 1190


## 2. Complejos simpliciales

En esta sección se mostrará cómo crear una función para calcular el **número de triángulos** dada una red **ponderada** (_weighted_) y un **umbral**. Dicha función, retornará una tupla con el número de tripletes y una lista con los tripletes encontrados.

In [ ]:
def compute_triplets(graph, th=None):
    """
    Calcula el número de triángulos dada una red ponderada (weighted) y un umbral
    
    PARAMETERS:
    graph: El grafo
    th: El umbral
    
    RETURNS:
    Una tupla formada por: el número de triángulos y una lista con los triángulos
    """
    if th==None:
        edge_list = list(graph.edges())
    else:
        edge_list = [(a,b) for a,b,c in graph.edges(data=True) if c['weight']<=th]
    
    ordered_list = [] # here we will store the edges as (a,b) where a<b always
    for a,b in edge_list:
        if a<b:
            ordered_list.append((a,b))
        else:
            ordered_list.append((b,a))

    ordered_list = set(ordered_list) # we want fast queries if possible

    triangles = set()
    for a,b in ordered_list:
        for c,d in ordered_list:
            if c>b:
                continue # as a<b<c<d it is impossible to form a triangle
            if a==c:
                # is a,b,d a triangle?
                x,y = (b,d) if b<d else (d,b) # ordered pair
                if (x,y) in ordered_list:
                    triangles.add((a,x,y))

    return (len(triangles), list(triangles))

Para analizar el umbral (_threshold_), emplearemos todos los valores entre 0 y 1 con intervalos de 0.1.

In [ ]:
triplets = []

for th in np.linspace(start=0, stop=1, num=11):
    a, _ = compute_triplets(G, th)
    triplets.append((th, a))
    print('th {:.2f}, triplets: {}'.format(th,a))

th 0.00, triplets: 0
th 0.10, triplets: 2
th 0.20, triplets: 17
th 0.30, triplets: 99
th 0.40, triplets: 365
th 0.50, triplets: 782
th 0.60, triplets: 1413
th 0.70, triplets: 2238
th 0.80, triplets: 3575
th 0.90, triplets: 5305
th 1.00, triplets: 6414
